In [5]:
import pandas as pd
import numpy as np
import sys
sys.path.append("..")
from data.scripts.simplified_finance_stats.fin_stats import fin_stats
from data.scripts.simplified_finance_stats.fin_stats_2 import fin_stats_2
from data.scripts.simplified_finance_stats.stock_stats import stock_stats
from value import valuation
from input_params import get_inp_params
from sklearn.linear_model import LinearRegression
from input_params import get_inp_params

import matplotlib.pyplot as plt
%matplotlib inline


In [14]:
# Set path for data
base_path = '../data/'
sheets_path = 'combined_simplified/combined_all_us.csv'
other_path = 'combined_simplified/others_all_us.csv'
mkt_path = 'combined_simplified/stock_stats_all_us.csv'

# setup all data
finances = fin_stats(base_path + sheets_path)
fin_others = fin_stats_2(base_path + other_path)
mkt_data = stock_stats(base_path + mkt_path)

Total Missing tickers: 181
Total Missing tickers: 181
Total Missing tickers: 181


In [ ]:
# Get the fundamental data
tick = 'GOOGL'
b = finances.get_sheet(tick,"balance_sheet")
i = finances.get_sheet(tick,"income_sheet")
c = finances.get_sheet(tick,"cashflow_sheet")
o = fin_others.get_sheet(tick)
mk = mkt_data.get_stock_data(tick)

inp_params = get_inp_params(tick,finances,fin_others,mkt_data)

In [ ]:
c.loc['capx']

In [ ]:
curr = b.loc['act'] - b.loc['lct']
curr = curr.values
print curr


In [60]:
%load_ext autoreload
%aimport value
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
v = value.valuation('FB',finances,fin_others,mkt_data)
v1,p1 = v.val_eq()

0.45292712931471163
12518.0
27638.0
Rev
[ 35986.03881997  46855.5970024   61008.29772997  79435.81194193
 103429.34410007 128919.44883116 153523.67224047 174287.70651666
 188169.69827663 192695.17952019]
ebit
[15568.79890282 19320.51668422 23918.2415384  29530.76175478
 36351.64596758 42694.36367345 47727.153487   50645.46742082
 50860.89823869 48173.79488005]
[12557.26735317 15397.44240142 18831.5320287  22966.39283394
 27921.42106929 32382.55956776 35740.73700731 37439.00030067
 37109.05072543 34685.13231363]
[1603.7550089  2088.16810217 2718.89783586 3540.13905017 4609.43560632
 4896.94454247 4726.75647342 3989.01163417 2666.89150573  869.39739603]
[0.04456603 0.04456603 0.04456603 0.04456603 0.04456603 0.03798453
 0.03078845 0.02288751 0.0141728  0.00451178]
[10953.51234428 13309.27429925 16112.63419284 19426.25378377
 23311.98546297 27485.61502529 31013.98053389 33449.98866651
 34442.15921969 33815.73491761]
0.18381530595941845
12518.0


In [48]:
v1

97.18689468437185

In [49]:
p1

1.1838015853232178

In [7]:
equity_list_path = '../equity_list/all_us_list.csv'

In [8]:
import os
os.path.abspath(equity_list_path)

'D:\\FA\\equity_list\\all_us_list.csv'

In [9]:
equity_list = pd.read_csv(equity_list_path)['tick'].tolist()

In [11]:
revenue = {}
for t in equity_list:
    try:
        i = finances.get_sheet(t,"income_sheet")
        revenue[t] = i.loc['revt']
    except:
        pass

AFP not found in income_sheet
CWTR not found in income_sheet
CTYN not found in income_sheet
CWE not found in income_sheet
CNF not found in income_sheet
CTC not found in income_sheet
DOW not found in income_sheet
DOYL not found in income_sheet
EK not found in income_sheet
HB not found in income_sheet
ICX not found in income_sheet
KLU not found in income_sheet
KES not found in income_sheet
KRN not found in income_sheet
MOB not found in income_sheet
MMED not found in income_sheet
NAS not found in income_sheet
GAS not found in income_sheet
OMAH not found in income_sheet
OVT not found in income_sheet
PICN not found in income_sheet
PINK not found in income_sheet
PITS not found in income_sheet
RJR not found in income_sheet
RLM not found in income_sheet
SAFC not found in income_sheet
SPC not found in income_sheet
SY not found in income_sheet
SNS not found in income_sheet
VSTA not found in income_sheet
WPO not found in income_sheet
WSC not found in income_sheet
Z not found in income_sheet
ACD n

In [13]:
import pickle
pickle_out = open("rev.pickle","wb")
pickle.dump(revenue, pickle_out)
pickle_out.close()